In [1]:
import gym
import torch
import torch.nn as nn


from models.td3 import TD3

In [2]:
class Actor(nn.Module):
    def __init__(self, n_sts: int, n_acts: int, dim: int):
        super(Actor, self).__init__()
                
        self.fc1 = nn.Linear(n_sts, dim)
        self.fc2 = nn.Linear(dim, n_acts)
        
        self.relu1 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = torch.tanh(self.fc2(x)) * 2
        
        return x

In [3]:
class Critic(nn.Module): #다름
    def __init__(self, n_sts: int, n_acts: int, dim: int):
        super(Critic, self).__init__()
                
        self.fc1 = nn.Linear(n_sts + n_acts, dim)
        self.fc2 = nn.Linear(dim, 1)
        
        self.relu1 = nn.ReLU()
                
    def forward(self, xs):
        x, a = xs
        
        x = self.relu1(self.fc1(torch.cat([x, a], 1)))
        x = self.fc2(x)
        
        return x

In [4]:
lr = 1e-3
tau = 5e-3
eps = 0
gamma = 0.99

n_sts = 3
n_acts = 1

dim = 256

n_epis = 500
n_epochs = 200
n_rollout = 10

In [5]:
act = Actor(n_sts, n_acts, dim)
act_opt = torch.optim.Adam(act.parameters(), lr=lr)

cri1 = Critic(n_sts, n_acts, dim)
cri_opt1 = torch.optim.Adam(cri1.parameters(), lr=lr)

cri2 = Critic(n_sts, n_acts, dim)
cri_opt2 = torch.optim.Adam(cri2.parameters(), lr=lr)

In [6]:
env = gym.make('Pendulum-v1', render_mode='human')

In [7]:
model = TD3(env, n_acts, act, act_opt, cri1, cri_opt1, cri2, cri_opt2, eps=eps, tau=tau, act_noise=0.2, act_range = (-2, 2))

In [8]:
model.train(n_epis, n_epochs, n_rollout)

c:\Users\esder\AppData\Local\Programs\Python\Python38\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


epoch: 20, score: -78.61484510217828, n_buffer: 210
epoch: 40, score: -74.41160686000048, n_buffer: 410
epoch: 60, score: -68.66533971374214, n_buffer: 610
epoch: 80, score: -63.63188993406493, n_buffer: 810
epoch: 100, score: -57.67259531507828, n_buffer: 1010
epoch: 120, score: -52.145870225257156, n_buffer: 1210
epoch: 140, score: -44.85566137722273, n_buffer: 1410
epoch: 160, score: -38.92058396531045, n_buffer: 1610
epoch: 180, score: -41.31992360065323, n_buffer: 1810
epoch: 20, score: -65.68585890240486, n_buffer: 2210
epoch: 40, score: -93.62856335830239, n_buffer: 2410
epoch: 60, score: -90.26485792149882, n_buffer: 2610
epoch: 80, score: -79.67363428289829, n_buffer: 2810
epoch: 100, score: -76.08621597172254, n_buffer: 3010
epoch: 120, score: -75.90480420152181, n_buffer: 3210
epoch: 140, score: -75.7077837406642, n_buffer: 3410
epoch: 160, score: -75.48588612674831, n_buffer: 3610
epoch: 180, score: -75.59984101835606, n_buffer: 3810
epoch: 20, score: -56.59060559225443, n_

KeyboardInterrupt: 

In [9]:
env.close()